# **1. Datenbereinigung**

In [16]:
# Libraries:
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from os import popen

In [17]:
# df Anpassung:
# ==============================================================================
df = pd.read_csv('/content/sample_data/sickness_table.csv')
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df = df.drop(columns=['Unnamed: 0', 'dafted', 'n_sby'])
df['calls'] = df['calls'].astype('int64')
df['n_sick'] = df['n_sick'].astype('int64')
df['sby_need'] = df['sby_need'].astype('int64')
df['year'] = df['year'].astype('int64')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1152 non-null   datetime64[ns]
 1   n_sick    1152 non-null   int64         
 2   calls     1152 non-null   int64         
 3   n_duty    1152 non-null   int64         
 4   sby_need  1152 non-null   int64         
 5   year      1152 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 54.1 KB


Hier wurden alle Datentypen vereinheitlicht.
Der Datentyp von date wurde zu einem datetime-Typ umgewandelt für die weitere Erstellung von zeitbasierten Features.

# **2. Feature Engineering**



In [18]:
# Zeitbasierte Features:
# ==============================================================================
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.isocalendar().week
df['weekday'] = df['date'].dt.weekday  # Montag=0, Sonntag=6
df['quarter'] = df['date'].dt.quarter  # Quartal


In [19]:
df.head()

,date,n_sick,calls,n_duty,sby_need,year,month,day,week,weekday,quarter
0,2016-04-01,73,8154,1700,4,2016,4,1,13,4,2
1,2016-04-02,64,8526,1700,70,2016,4,2,13,5,2
2,2016-04-03,68,8088,1700,0,2016,4,3,13,6,2
3,2016-04-04,71,7044,1700,0,2016,4,4,14,0,2
4,2016-04-05,63,7236,1700,0,2016,4,5,14,1,2


In [20]:
# Jahreszeiten:

def get_season(month):
  if month in [12,1,2]:
    return '4' # Winter
  elif month in [3,4,5]:
    return '1' # Frühling
  elif month in [6,7,8]:
    return '2' # Sommer
  else:
    return '3' # Herbst

df['season'] = df['month'].apply(get_season)

df.head()

,date,n_sick,calls,n_duty,sby_need,year,month,day,week,weekday,quarter,season
0,2016-04-01,73,8154,1700,4,2016,4,1,13,4,2,1
1,2016-04-02,64,8526,1700,70,2016,4,2,13,5,2,1
2,2016-04-03,68,8088,1700,0,2016,4,3,13,6,2,1
3,2016-04-04,71,7044,1700,0,2016,4,4,14,0,2,1
4,2016-04-05,63,7236,1700,0,2016,4,5,14,1,2,1


In [21]:
# Schulferien hinzufügen:

dfb = pd.read_excel('/content/sample_data/holidays_berlin.xlsx')
dfb['Datum Begin'] = pd.to_datetime(dfb['Datum Begin'], format='%d.%m.%Y')
dfb['Datum Ende'] = pd.to_datetime(dfb['Datum Ende'], format='%d.%m.%Y')  # Korrigierte Zeile
date_range = pd.date_range(start='2016-04-01', end='2019-05-27')
days_dfb = pd.DataFrame(date_range, columns=['date'])
days_dfb['is_holiday'] = 0

for _, row in dfb.iterrows():
    days_dfb.loc[(days_dfb['date'] >= row['Datum Begin']) & (days_dfb['date'] <= row['Datum Ende']), 'is_holiday'] = 1

df = pd.merge(df, days_dfb, on='date', how='left')
df['is_holiday'] = df['is_holiday'].fillna(0)

print(df.head())

        date  n_sick  calls  n_duty  sby_need  year  month  day  week  \
0 2016-04-01      73   8154    1700         4  2016      4    1    13   
1 2016-04-02      64   8526    1700        70  2016      4    2    13   
2 2016-04-03      68   8088    1700         0  2016      4    3    13   
3 2016-04-04      71   7044    1700         0  2016      4    4    14   
4 2016-04-05      63   7236    1700         0  2016      4    5    14   

   weekday  quarter season  is_holiday  
0        4        2      1           1  
1        5        2      1           1  
2        6        2      1           0  
3        0        2      1           0  
4        1        2      1           0  


In [22]:
# Einwohner Berlin:

population_df = pd.read_excel('/content/sample_data/Einwohner.xlsx')
population_df.head()
df = pd.merge(df, population_df, left_on='year', right_on='year', how='left')

df.head()


,date,n_sick,calls,n_duty,sby_need,year,month,day,week,weekday,quarter,season,is_holiday,Einwohner
0,2016-04-01,73,8154,1700,4,2016,4,1,13,4,2,1,1,3574830
1,2016-04-02,64,8526,1700,70,2016,4,2,13,5,2,1,1,3574830
2,2016-04-03,68,8088,1700,0,2016,4,3,13,6,2,1,0,3574830
3,2016-04-04,71,7044,1700,0,2016,4,4,14,0,2,1,0,3574830
4,2016-04-05,63,7236,1700,0,2016,4,5,14,1,2,1,0,3574830


In [23]:
# Einige Lag-Features

def create_lags(df, column, lags):
  for lag in lags:
    df[f'lag_{lag}'] = df[column].shift(lag)
  return df

lags =[1,7,31,365]
df = create_lags(df, 'calls', lags)

df.info(110)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1152 non-null   datetime64[ns]
 1   n_sick      1152 non-null   int64         
 2   calls       1152 non-null   int64         
 3   n_duty      1152 non-null   int64         
 4   sby_need    1152 non-null   int64         
 5   year        1152 non-null   int32         
 6   month       1152 non-null   int32         
 7   day         1152 non-null   int32         
 8   week        1152 non-null   UInt32        
 9   weekday     1152 non-null   int32         
 10  quarter     1152 non-null   int32         
 11  season      1152 non-null   object        
 12  is_holiday  1152 non-null   int64         
 13  Einwohner   1152 non-null   int64         
 14  lag_1       1151 non-null   float64       
 15  lag_7       1145 non-null   float64       
 16  lag_31      1121 non-nul

In [24]:
# Diff-Features:

def create_diffs(df, columns, diffs):
  for diff in diffs:
    df[f'diff_{diff}'] = df[columns].diff(diff)
  return df

diffs=[1,7,31,365]
df = create_diffs(df, 'calls', diffs)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1152 non-null   datetime64[ns]
 1   n_sick      1152 non-null   int64         
 2   calls       1152 non-null   int64         
 3   n_duty      1152 non-null   int64         
 4   sby_need    1152 non-null   int64         
 5   year        1152 non-null   int32         
 6   month       1152 non-null   int32         
 7   day         1152 non-null   int32         
 8   week        1152 non-null   UInt32        
 9   weekday     1152 non-null   int32         
 10  quarter     1152 non-null   int32         
 11  season      1152 non-null   object        
 12  is_holiday  1152 non-null   int64         
 13  Einwohner   1152 non-null   int64         
 14  lag_1       1151 non-null   float64       
 15  lag_7       1145 non-null   float64       
 16  lag_31      1121 non-nul

In [25]:
# Moving Avrg:

df['moving_avg_7'] = df['calls'].rolling(window=7).mean()
df['moving_avg_31'] = df['calls'].rolling(window=31).mean()
df['moving_avg_365'] = df['calls'].rolling(window=365).mean()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1152 non-null   datetime64[ns]
 1   n_sick          1152 non-null   int64         
 2   calls           1152 non-null   int64         
 3   n_duty          1152 non-null   int64         
 4   sby_need        1152 non-null   int64         
 5   year            1152 non-null   int32         
 6   month           1152 non-null   int32         
 7   day             1152 non-null   int32         
 8   week            1152 non-null   UInt32        
 9   weekday         1152 non-null   int32         
 10  quarter         1152 non-null   int32         
 11  season          1152 non-null   object        
 12  is_holiday      1152 non-null   int64         
 13  Einwohner       1152 non-null   int64         
 14  lag_1           1151 non-null   float64       
 15  lag_

In [26]:
# Wetterdaten
dfw = pd.read_csv('sample_data/berlin_daily_weather.csv')
dfw = dfw.drop(['STATION','Unnamed: 0', 'NAME', 'LATITUDE', 'LONGITUDE','ELEVATION',], axis=1)
dfw['DATE'] = pd.to_datetime(dfw['DATE'])
dfw = dfw.rename(columns={'DATE': 'date'})

start_date = '2016-04-01'
end_date = '2019-05-27'

dfw = dfw[(dfw['date'] >= start_date) & (dfw['date'] <= end_date)]
df = pd.merge(dfw, df, on='date', how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1152 non-null   datetime64[ns]
 1   PRCP            1152 non-null   float64       
 2   SNWD            1152 non-null   float64       
 3   TMAX            1152 non-null   float64       
 4   TMIN            1152 non-null   float64       
 5   n_sick          1152 non-null   int64         
 6   calls           1152 non-null   int64         
 7   n_duty          1152 non-null   int64         
 8   sby_need        1152 non-null   int64         
 9   year            1152 non-null   int32         
 10  month           1152 non-null   int32         
 11  day             1152 non-null   int32         
 12  week            1152 non-null   UInt32        
 13  weekday         1152 non-null   int32         
 14  quarter         1152 non-null   int32         
 15  seas

In [27]:
# Datengenerierung und entfernen von NaN's:
# ==============================================================================

df.bfill(inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1152 non-null   datetime64[ns]
 1   PRCP            1152 non-null   float64       
 2   SNWD            1152 non-null   float64       
 3   TMAX            1152 non-null   float64       
 4   TMIN            1152 non-null   float64       
 5   n_sick          1152 non-null   int64         
 6   calls           1152 non-null   int64         
 7   n_duty          1152 non-null   int64         
 8   sby_need        1152 non-null   int64         
 9   year            1152 non-null   int32         
 10  month           1152 non-null   int32         
 11  day             1152 non-null   int32         
 12  week            1152 non-null   UInt32        
 13  weekday         1152 non-null   int32         
 14  quarter         1152 non-null   int32         
 15  seas

In [28]:
# Änderung der Datentypen für einheitliche Daten:
float_cols = df.select_dtypes(include=['float64']).columns
df[float_cols] = df[float_cols].astype('int')

float_cols = df.select_dtypes(include=['int32']).columns
df[float_cols] = df[float_cols].astype('int')

df['week'] = df['week'].astype('int')
df['season'] = df['season'].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1152 non-null   datetime64[ns]
 1   PRCP            1152 non-null   int64         
 2   SNWD            1152 non-null   int64         
 3   TMAX            1152 non-null   int64         
 4   TMIN            1152 non-null   int64         
 5   n_sick          1152 non-null   int64         
 6   calls           1152 non-null   int64         
 7   n_duty          1152 non-null   int64         
 8   sby_need        1152 non-null   int64         
 9   year            1152 non-null   int64         
 10  month           1152 non-null   int64         
 11  day             1152 non-null   int64         
 12  week            1152 non-null   int64         
 13  weekday         1152 non-null   int64         
 14  quarter         1152 non-null   int64         
 15  seas

In [29]:
# Erstellung der FT-Terme:
# Müssen float64 Datentypen bleiben.

df['day_of_year'] = df['date'].dt.dayofyear
df['sin_year'] = np.sin(2 * np.pi * df['day_of_year'] / 365.25)
df['cos_year'] = np.cos(2 * np.pi * df['day_of_year'] / 365.25)

df['day_of_month'] = df['date'].dt.day  # Korrektur hier
df['sin_month'] = np.sin(2 * np.pi * df['day_of_month'] / 30.5)
df['cos_month'] = np.cos(2 * np.pi * df['day_of_month'] / 30.5)

df['weekday'] = df['date'].dt.weekday
df['sin_week'] = np.sin(2 * np.pi * df['weekday'] / 7)
df['cos_week'] = np.cos(2 * np.pi * df['weekday'] / 7)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            1152 non-null   datetime64[ns]
 1   PRCP            1152 non-null   int64         
 2   SNWD            1152 non-null   int64         
 3   TMAX            1152 non-null   int64         
 4   TMIN            1152 non-null   int64         
 5   n_sick          1152 non-null   int64         
 6   calls           1152 non-null   int64         
 7   n_duty          1152 non-null   int64         
 8   sby_need        1152 non-null   int64         
 9   year            1152 non-null   int64         
 10  month           1152 non-null   int64         
 11  day             1152 non-null   int64         
 12  week            1152 non-null   int64         
 13  weekday         1152 non-null   int32         
 14  quarter         1152 non-null   int64         
 15  seas

# **4. Speichern des neuen DataFrames als Excel-Datei**

In [30]:
df.to_excel('df_features.xlsx')

# **4. Fazit:**

Die genannten Features bieten eine erste Orientierung bei der Erstellung der Modelle. Während des Entwicklungsprozesses können jedoch zusätzliche Features hinzukommen, was das Feature Engineering zu einem dynamischen und anpassungsfähigen Prozess macht. Diese Flexibilität ermöglicht es, die Modelle kontinuierlich zu optimieren und an neue Erkenntnisse oder Daten anzupassen.